In [1]:
import sys

In [3]:
sys.getrefcount(1), sys.getrefcount(257)

(4294967295, 3)

In [4]:
lst = [1, 2, 3]
sys.getrefcount(lst)  # obj = lst

2

In [5]:
lst2 = lst

In [6]:
sys.getrefcount(lst)

3

In [7]:
del lst2

In [8]:
sys.getrefcount(lst)

2

In [19]:
s1 = "qwerty123"

In [20]:
s2 = "qwerty123"

In [21]:
s1 is s2

True

In [24]:
sys.getrefcount(s1), sys.getrefcount(s2), sys.getrefcount("sys")

(3, 3, 4294967295)

In [12]:
s1 = "qwerty123!"

In [13]:
s2 = "qwerty123!"

In [14]:
s1 is s2

False

In [18]:
sys.getrefcount(s1)

7

In [25]:
lst = [1, 2, 3]

def count_refs(obj):
    print(f"{sys.getrefcount(obj)=}")

print("before", sys.getrefcount(lst))

count_refs(lst)

print("after", sys.getrefcount(lst))

before 2
sys.getrefcount(obj)=3
after 2


In [26]:
class Animal:
    def __init__(self, name, food):
        self.name = name
        self.food = food

    def __str__(self):
        return f"Animal({self.name}, {id(self)})"

    def __del__(self):
        print(f"del {str(self)=}")

In [33]:
tiger = Animal("tiger", "meat")
print("created", tiger)

sys.getrefcount(tiger)

del tiger

del str(self)='Animal(tiger, 4552684992)'
created Animal(tiger, 4552685904)
del str(self)='Animal(tiger, 4552685904)'


In [35]:
tiger = Animal("tiger", "meat")
hog = Animal("hog", tiger)
tiger.food = hog

print("created", tiger)
print("created", hog)

print(sys.getrefcount(tiger), sys.getrefcount(tiger))

created Animal(tiger, 4552725088)
created Animal(hog, 4551315536)
3 3


In [49]:
tiger = Animal("tiger", "meat")
hog = Animal("hog", tiger)
tiger.food = hog

print("created", tiger, tiger.__dict__)
print("created", hog, hog.__dict__)

print(sys.getrefcount(tiger), sys.getrefcount(tiger))

del tiger
del hog

created Animal(tiger, 4559270896) {'name': 'tiger', 'food': <__main__.Animal object at 0x110136630>}
created Animal(hog, 4564674096) {'name': 'hog', 'food': <__main__.Animal object at 0x10fc0f3f0>}
3 3


In [37]:
import gc

In [42]:
gc.collect()

del str(self)='Animal(tiger, 4554609424)'
del str(self)='Animal(hog, 4555459952)'


4455

In [39]:
import ctypes

In [40]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]

In [47]:
tiger = Animal("tiger", "meat")
hog = Animal("hog", tiger)
tiger.food = hog

id_tiger = id(tiger)
id_hog = id(hog)

print("created", tiger, id_tiger)
print("created", hog, id_hog)

print(sys.getrefcount(tiger), sys.getrefcount(tiger))

for name, addr in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"{name}: {PyObject.from_address(addr).refcnt=}")

del tiger
del hog

for name, addr in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"{name}: {PyObject.from_address(addr).refcnt=}")

gc.collect()

for name, addr in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"{name}: {PyObject.from_address(addr).refcnt=}")

created Animal(tiger, 4554238016) 4554238016
created Animal(hog, 4564596048) 4564596048
3 3
tiger: PyObject.from_address(addr).refcnt=2
hog: PyObject.from_address(addr).refcnt=2
tiger: PyObject.from_address(addr).refcnt=1
hog: PyObject.from_address(addr).refcnt=1
del str(self)='Animal(tiger, 4554238016)'
del str(self)='Animal(hog, 4564596048)'
tiger: PyObject.from_address(addr).refcnt=4515801808
hog: PyObject.from_address(addr).refcnt=0


In [46]:
tiger = Animal("tiger", "meat")
hog = Animal("hog", tiger)
tiger.food = hog

id_tiger = id(tiger)
id_hog = id(hog)

print("created", tiger, id_tiger)
print("created", hog, id_hog)

print(sys.getrefcount(tiger), sys.getrefcount(tiger))

for name, addr in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"{name}: {PyObject.from_address(addr).refcnt=}")

del tiger
del hog

for name, addr in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"{name}: {PyObject.from_address(addr).refcnt=}")

gc.collect()

for i in range(1000000):
    x = 1000

for name, addr in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"{name}: {PyObject.from_address(addr).refcnt=}")

created Animal(tiger, 4554158192) 4554158192
created Animal(hog, 4554156432) 4554156432
3 3
tiger: PyObject.from_address(addr).refcnt=2
hog: PyObject.from_address(addr).refcnt=2
tiger: PyObject.from_address(addr).refcnt=1
hog: PyObject.from_address(addr).refcnt=1
del str(self)='Animal(tiger, 4554158192)'
del str(self)='Animal(hog, 4554156432)'
tiger: PyObject.from_address(addr).refcnt=-1082298007296
hog: PyObject.from_address(addr).refcnt=-16645888


In [50]:
import weakref

In [51]:
tiger = Animal("tiger", "meat")
print(1, sys.getrefcount(tiger))

weakref.finalize(tiger, lambda: print("tiger deleted"))

weak = weakref.ref(tiger)
print(2, sys.getrefcount(tiger))

print(weak)

tiger_new = weak()
print(3, sys.getrefcount(tiger))

del tiger_new, tiger
print(4, "del")

tiger_new = weak()
print(5, tiger_new)

1 2
2 2
<weakref at 0x10faa36a0; to 'Animal' at 0x10fc50350>
3 3
del str(self)='Animal(tiger, 4559536976)'
tiger deleted
4 del
5 None


In [67]:
class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.z = 33


class Point2D(Point):
    __slots__ = ("x", "y")

    def __init__(self, x, y):
        #super().__init__(x, y)
        self.x = x
        self.y = y
        self.z = 33

In [68]:
p = Point(10, 20)
ps = Point2D(30, 40)

In [65]:
p.__dict__

{'x': 10, 'y': 20, 'z': 33}

In [70]:
ps.__dict__

{'z': 33, 'xxx': 34}

In [77]:
class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y


class Point2D:
    __slots__ = ("x", "y")

    def __init__(self, x, y):
        self.x = x
        self.y = y

In [78]:
p = Point(10, 20)
ps = Point2D(30, 40)

In [79]:
print(p.x, p.y, ps.x, ps.y)

10 20 30 40


In [80]:
p.x = 99
ps.x = 42

In [81]:
print(p.x, p.y, ps.x, ps.y)

99 20 42 40


In [82]:
p.z = 25

In [83]:
ps.z = 25

AttributeError: 'Point2D' object has no attribute 'z' and no __dict__ for setting new attributes

In [85]:
Point.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 1,
              '__init__': <function __main__.Point.__init__(self, x, y)>,
              '__static_attributes__': ('x', 'y'),
              '__dict__': <attribute '__dict__' of 'Point' objects>,
              '__weakref__': <attribute '__weakref__' of 'Point' objects>,
              '__doc__': None})

In [86]:
Point2D.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 8,
              '__slots__': ('x', 'y'),
              '__init__': <function __main__.Point2D.__init__(self, x, y)>,
              '__static_attributes__': ('x', 'y'),
              'x': <member 'x' of 'Point2D' objects>,
              'y': <member 'y' of 'Point2D' objects>,
              '__doc__': None})

In [88]:
def create_objs(cls, n):
    objs = [cls(2 ** 20, 2 ** 21) for _ in range(n)]
    for obj in objs:
        obj.x = obj.x + obj.y
        obj.y = obj.x - obj.y
        obj.x = obj.x + obj.x + obj.x - obj.y

In [90]:
%%time

create_objs(Point, 5_000_000)

CPU times: user 5.39 s, sys: 534 ms, total: 5.93 s
Wall time: 7.06 s


In [91]:
%%time

create_objs(Point2D, 5_000_000)

CPU times: user 4.83 s, sys: 440 ms, total: 5.27 s
Wall time: 6.49 s


In [1]:
class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y


class Point2D(Point):
    __slots__ = ("x", "y")

    def __init__(self, x, y):
        self.x = x
        self.y = y

In [2]:
p = Point2D(10, 20)

In [3]:
p.__dict__

{}

In [4]:
import sys

In [5]:
sys.getsizeof(p)

64

In [6]:
sys.getsizeof(p.__dict__)

304

In [7]:
sys.getsizeof({})

64

In [8]:
sys.getsizeof({1: 11})

224

In [9]:
dct = {}

for i in range(8):
    dct[i] = i
    print(i, sys.getsizeof(dct), len(dct))

0 224 1
1 224 2
2 224 3
3 224 4
4 224 5
5 352 6
6 352 7
7 352 8
